# Topic Modelling

## Durchführung Topic Modelling

In [ ]:
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Projekt_Mobilisierung/Data/final_df_pm.csv')

In [ ]:
df_description = df[['id', 'body']].copy()
df_ocr_text = df[['id', 'ocr_text']].copy()

df_description.rename(columns={'body': 'Text'}, inplace=True)
df_ocr_text.rename(columns={'ocr_text': 'Text'}, inplace=True)

df_description['Text Type'] = 'Caption'
df_ocr_text['Text Type'] = 'OCR'

new_df = pd.concat([df_description, df_ocr_text])

new_df.dropna(subset=['Text'], inplace=True)
new_df = new_df[new_df['Text'].str.strip() != '']

new_df.reset_index(drop=True, inplace=True)

In [ ]:
!pip install -q bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.5 MB/s eta 0:00:00


In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

STOPWORDS = stopwords.words('german')

from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=STOPWORDS)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
docs = new_df['Text']

In [ ]:
docs = [str(doc) for doc in docs if pd.notna(doc)]
docs = [str(doc) for doc in docs]
docs

["Am Sonntag wird gewählt und es geht um nicht weniger als Bayern Zukunft!\n💚 konsequenten Klimaschutz\n💚 billigen Strom\n💚 die besten Startchancen für Bayerns Kinder gibt's nur mit uns Grünen 🌻\nAlles Stimmen grün!\n\n#TeamBayern #ltw23 #MitDir #allestimmengrün",
 'Stand am Schrannenplatz heute mit unserer Direktkandidatin @marina.abstreiter unserem MdB @leckert_ , Mona, Klaus und yourstruly. Super Stimmung und viele gute Gespräche. Leon: “Mit einer der besten Stände, an dem ich bisher war!” #neuburg #neuburganderdonau #neuburgfitfürdiezukunft #ltw2023bayern #grueneenergie #gruenebayern Foto: @chris_eschner',
 'Grün klingelt👋\nWir sind im Landkreis unterwegs und erinnern euch dran: Am 08. Oktober ist Landtagswahl und Bezirkstagswahl. \nGeht wählen! Am besten natürlich grün 💚\n\nDanke für die vielen netten Gespräche an euren Haustüren :)\n\n#TeamBayern #MitDir #ltw23',
 "Toller Abend mit @kathaschulze im Bootshaus in Neuburg.\nGrüne Energie für Bayern 💪\nWir wollen \n🌻Sauberen, günstig

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(language="multilingual", calculate_probabilities=True, verbose=True, vectorizer_model=vectorizer_model)
topics, probs = topic_model.fit_transform(docs)

In [ ]:
freq = topic_model.get_topic_info(); freq.head(678)

In [ ]:
len(freq)

743

In [ ]:
freq.to_csv('/content/drive/MyDrive/Projekt_Mobilisierung/topic_model.csv', index=False)

In [ ]:
topic_model.save("/content/drive/MyDrive/Projekt_Mobilisierung/Output/topic_model")

## Analyse Topic Modelling

In [ ]:
from bertopic import BERTopic
topic_model_loaded = BERTopic.load("/content/drive/MyDrive/Projekt_Mobilisierung/Output/topic_model")

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(top_n_topics=15)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Projekt_Mobilisierung/Output/topic_model_annoted.csv', delimiter=';')

In [ ]:
df.head(12)

In [ ]:
import matplotlib.pyplot as plt

cat_counts = df['Kat'].value_counts()

if 'Sonstiges' in cat_counts.index:
    sonstiges_count = cat_counts['Sonstiges']
    cat_counts = cat_counts.drop('Sonstiges')
    cat_counts['Sonstiges'] = sonstiges_count

ax = cat_counts.plot(kind='bar', color='black')

for i, count in enumerate(cat_counts):
    ax.text(i, count + 3, str(count), ha='center', color='black')

plt.xlabel('Kommunikationskategorie', labelpad=20)
plt.ylabel('Anzahl Topics')
plt.title('Anzahl Topics nach Kategorie')

plt.ylim(0, 250)

ax.set_xticklabels([tick.get_text().capitalize() for tick in ax.get_xticklabels()], rotation=90, fontsize=10)

new_labels = []
for tick in plt.gca().get_xticklabels():
    label = tick.get_text()
    if label == 'Negative campainging':
        label = 'Negative Campaigning'
    elif label == 'Information zur wahl':
        label = 'Informationen zur Wahl'
    new_labels.append(label)

plt.gca().set_xticklabels(new_labels)

plt.show()

Michael Achmann. (2023). michaelachmann/social-media-lab: 2023-11-27 (v0.0.12). Zenodo. https://doi.org/10.5281/zenodo.8199901

Grootendorst, Maarten. 2022. “BERTopic: Neural topic modeling with a class-based TF-IDF procedure,” March. http://arxiv.org/abs/2203.05794.